# Sentiment Analysis on Turkish Reviews

This notebook performs sentiment analysis on Turkish restaurant reviews using a pre-trained BERT model. We'll analyze the sentiment of each review and classify them as positive or negative.

## Setup and Data Loading
First, we'll import the necessary libraries and load our cleaned dataset.


In [5]:
# Import necessary libraries
import pandas as pd
import numpy as np
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import pipeline
import torch
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings('ignore')

# Read the data and sample first 1000 rows
df = pd.read_excel('CleanedDataFrame.xlsx')
df_sample = df
df_sample


,Mekan_Adı,Kullanıcı_Adı,Yorum,Puan,Yorum_Tarihi,Restoran_Kategorisi,Restoran_Toplam_Yorum_Sayısı,Ortalama_Restoran_Puanı,Restoran_Adresi
0,01_adanali_ismail_ustanin_yeri,güvenç tünay,Mezeler etler tavuk harika\nHizmet süper\nHarika,5,2025-07-13,Restoran,767,4.1,"Sancak, Tiflis Cd. No:41, 06550 Çankaya/Ankara"
1,01_adanali_ismail_ustanin_yeri,Ali Can Vurgun,Mezeleri temizliği yemeklerin lezzeti harikadi...,5,2025-06-20,Restoran,767,4.1,"Sancak, Tiflis Cd. No:41, 06550 Çankaya/Ankara"
2,01_adanali_ismail_ustanin_yeri,Pelin Yildirim,Resmen dolandırıldık. Yani bunu başka türlü aç...,1,2025-06-20,Restoran,767,4.1,"Sancak, Tiflis Cd. No:41, 06550 Çankaya/Ankara"
3,01_adanali_ismail_ustanin_yeri,Berkay Akın,Muhtemelen ilerleyen aylarda oturduğunuz masad...,1,2025-06-20,Restoran,767,4.1,"Sancak, Tiflis Cd. No:41, 06550 Çankaya/Ankara"
4,01_adanali_ismail_ustanin_yeri,Murat Engin,"Duman kokusu tüm mahalleyi sarıyor, otopark de...",1,2025-06-20,Restoran,767,4.1,"Sancak, Tiflis Cd. No:41, 06550 Çankaya/Ankara"
...,...,...,...,...,...,...,...,...,...
153481,zumrut_aile_restaurant,Erkan Badem,26 8 2023 evet güzel bir ortam ve tavsiye ile ...,1,2024-07-20,Restoran,504,3.3,"Yakacık Çarşı, Şekersuyu Sokak, Yakacık No:16,..."
153482,zumrut_aile_restaurant,Dicle Diren,Tülay Maciran kendisini ilk kez dinlemek için ...,1,2024-07-20,Restoran,504,3.3,"Yakacık Çarşı, Şekersuyu Sokak, Yakacık No:16,..."
153483,zumrut_aile_restaurant,Büşra Şaran,23.08.2023 Tülay Maciran için 2 kişi gittik. N...,1,2024-07-20,Restoran,504,3.3,"Yakacık Çarşı, Şekersuyu Sokak, Yakacık No:16,..."
153484,zumrut_aile_restaurant,ahmet şen,"Tam bir kaptı kaçtıcı mekan,cüzdaniniz şişkin ...",1,2024-07-20,Restoran,504,3.3,"Yakacık Çarşı, Şekersuyu Sokak, Yakacık No:16,..."


## Sentiment Analysis Model Setup

Here we:
1. Load the Turkish BERT model specifically trained for sentiment analysis (`savasy/bert-base-turkish-sentiment-cased`)
2. Create a sentiment analysis pipeline
3. Define a function that:
   - Analyzes the sentiment of each review
   - Returns a sentiment score (-1 to 1) and label (positive/negative)
   - Handles potential errors for long texts
4. Apply the sentiment analysis to our dataset

In [ ]:
# 1. Sentiment Analysis using Turkish BERT
# We'll use savasy/bert-base-turkish-sentiment-cased
from transformers import pipeline, AutoTokenizer, AutoModelForSequenceClassification

# Load model
sentiment_model_name = "savasy/bert-base-turkish-sentiment-cased"
tokenizer = AutoTokenizer.from_pretrained(sentiment_model_name)
model = AutoModelForSequenceClassification.from_pretrained(sentiment_model_name)

sentiment_pipeline = pipeline(
    "sentiment-analysis",
    model=model,
    tokenizer=tokenizer
)

# Sentiment function
def detailed_sentiment_analysis(text):
    try:
        result = sentiment_pipeline(text)[0]
        label = result['label'].lower()
        confidence = result['score']
        score = 1.0 if label == 'positive' else -1.0 if label == 'negative' else 0.0
        final_score = score * confidence
        return final_score, label
    except Exception as e:
        print(f"Error processing text: {e}")
        return None, None

# Apply sentiment analysis
df_sample[['sentiment_score', 'label']] = df_sample['Yorum'].apply(
    lambda x: pd.Series(detailed_sentiment_analysis(x))
)   


In [7]:
df_sample

,Mekan_Adı,Kullanıcı_Adı,Yorum,Puan,Yorum_Tarihi,Restoran_Kategorisi,Restoran_Toplam_Yorum_Sayısı,Ortalama_Restoran_Puanı,Restoran_Adresi,sentiment_score,label
0,01_adanali_ismail_ustanin_yeri,güvenç tünay,Mezeler etler tavuk harika\nHizmet süper\nHarika,5,2025-07-13,Restoran,767,4.1,"Sancak, Tiflis Cd. No:41, 06550 Çankaya/Ankara",0.984305,positive
1,01_adanali_ismail_ustanin_yeri,Ali Can Vurgun,Mezeleri temizliği yemeklerin lezzeti harikadi...,5,2025-06-20,Restoran,767,4.1,"Sancak, Tiflis Cd. No:41, 06550 Çankaya/Ankara",0.955563,positive
2,01_adanali_ismail_ustanin_yeri,Pelin Yildirim,Resmen dolandırıldık. Yani bunu başka türlü aç...,1,2025-06-20,Restoran,767,4.1,"Sancak, Tiflis Cd. No:41, 06550 Çankaya/Ankara",-0.999175,negative
3,01_adanali_ismail_ustanin_yeri,Berkay Akın,Muhtemelen ilerleyen aylarda oturduğunuz masad...,1,2025-06-20,Restoran,767,4.1,"Sancak, Tiflis Cd. No:41, 06550 Çankaya/Ankara",0.800201,positive
4,01_adanali_ismail_ustanin_yeri,Murat Engin,"Duman kokusu tüm mahalleyi sarıyor, otopark de...",1,2025-06-20,Restoran,767,4.1,"Sancak, Tiflis Cd. No:41, 06550 Çankaya/Ankara",-0.950211,negative
...,...,...,...,...,...,...,...,...,...,...,...
153481,zumrut_aile_restaurant,Erkan Badem,26 8 2023 evet güzel bir ortam ve tavsiye ile ...,1,2024-07-20,Restoran,504,3.3,"Yakacık Çarşı, Şekersuyu Sokak, Yakacık No:16,...",-0.998783,negative
153482,zumrut_aile_restaurant,Dicle Diren,Tülay Maciran kendisini ilk kez dinlemek için ...,1,2024-07-20,Restoran,504,3.3,"Yakacık Çarşı, Şekersuyu Sokak, Yakacık No:16,...",-0.999044,negative
153483,zumrut_aile_restaurant,Büşra Şaran,23.08.2023 Tülay Maciran için 2 kişi gittik. N...,1,2024-07-20,Restoran,504,3.3,"Yakacık Çarşı, Şekersuyu Sokak, Yakacık No:16,...",-0.998674,negative
153484,zumrut_aile_restaurant,ahmet şen,"Tam bir kaptı kaçtıcı mekan,cüzdaniniz şişkin ...",1,2024-07-20,Restoran,504,3.3,"Yakacık Çarşı, Şekersuyu Sokak, Yakacık No:16,...",-0.999215,negative


## Testing the Sentiment Analysis

Let's test our sentiment analysis model on the first 5 reviews to verify that it's working correctly. This will help us:
- See the actual review text
- Check the sentiment scores and labels
- Verify that our model is producing reasonable results

In [8]:
print("Testing first 5 reviews:")
for i in range(min(5, len(df_sample))):
    review = df_sample['Yorum'].iloc[i]
    print("\n--- Review #{} ---".format(i+1))
    print("Yorum:", review)
    score, label = detailed_sentiment_analysis(review)
    print(f"Returned score: {score}, label: {label}")


Testing first 5 reviews:

--- Review #1 ---
Yorum: Mezeler etler tavuk harika
Hizmet süper
Harika
Returned score: 0.9843053221702576, label: positive

--- Review #2 ---
Yorum: Mezeleri temizliği yemeklerin lezzeti harikadir çalışanların ilgisi alakası gayet güzeldir tavsiye ediyorum ve hayırlı işler diliyorum semtinizde böyle güzel bir mekanın olması bizleri mutlu ediyor 🧿
Returned score: 0.9555625915527344, label: positive

--- Review #3 ---
Yorum: Resmen dolandırıldık. Yani bunu başka türlü açıklayamam bi büyük bi küçük rakı 3 adana ve 4 lokuma 25k hesap çıkarmışlar. Gidip sorduğumuzda da bu kadar tuttunuz burda bi de hesap mı soruyorsunuz diye bir dönüş oldu. Siz açık tutarsanız kalırız kapatırsanız kalkarız, bunun için dolandırılmamıza gerek yoktu. Sakın sakın sakın gitmeyin!!!!!!!
Returned score: -0.9991747736930847, label: negative

--- Review #4 ---
Yorum: Muhtemelen ilerleyen aylarda oturduğunuz masadan, sandalyeden ve masa örtüsünden de para alacak mekandır. Racon der ki misaf

## Distribution of Sentiment Labels

Let's check the distribution of positive and negative reviews in our dataset to understand the overall sentiment balance.

In [9]:
df_sample['label'].value_counts()

label
positive    91536
negative    61876
Name: count, dtype: int64

In [10]:
df_sample.to_csv('son_sentiment.csv', index=False)